## Key Expiration
Redis provides an option to proactively evict a key after a set duration, using the `EXPIRE` command:

```bash
$ SET game valorant
OK
$ EXPIRE game 10
OK
```
Key *game* would expire in 10 seconds.

We can add `EX` option to specify a timeout while defining a value (only available for `SET`):
```bash
$ SET game valorant EX 10
```

Expiration is done by associating a key with Unix timestamp and then checking the system time. if you set a key with a time to live of 1000 seconds, and then set your computer time 2000 seconds in the future, the key will be expired immediately, instead of lasting for 1000 seconds.

## Key Eviction
When all allocated memory is consumed, Redis needs to free up some memory before it can add new data. Eviction policies provided by Redis are:

### Eviction Policies
Redis evicts data when it runs out of memory allocated to it 
<div style="display:inline-block" />
    
| Policy          | Description                                                                                     |
|-----------------|-------------------------------------------------------------------------------------------------|
| noeviction      | Returns an error if the memory limit has been reached when trying to insert more data (default) |
| allkeys-lru     | Evicts the least recently used keys out of all keys                                             |
| allkeys-lfu     | Evicts the least frequently used keys out of all keys                                           |
| allkeys-random  | Randomly evicts keys out of all keys                                                            |
| volatile-lru    | Evicts the least recently used keys out of all keys with an “expire” field set                  |
| volatile-lfu    | Evicts the least frequently used keys out of all keys with an “expire” field set                |
| volatile-random | Randomly evicts keys with an “expire” field set                                                 |
| volatile-ttl    | Evicts the shortest time-to-live keys out of all keys with an “expire” field set.               |
</div>

What happens when volatile-* policy is selected, but there are no keys with expiry set? The behaviour would be similar to noeviction. Why choose volatile-* instead of allkeys-*? Because some keys represent critical data (like a user's session state) that should never be deleted unless explicitly told to. Theys keys are set without expiration time.

To view the current eviction policy:
```bash
$ CONFIG GET maxmemory-policy
noeviction
```

To set this value at runtime,
```bash
$ CONFIG SET maxmemory-policy allkeys-lru
OK
```

Or set the `maxmemory-policy` setting in Redis.conf.

**Approximation:** the LRU and LFU algorithms used by Redis are not 100% accurate. This is done to save memory. LRU samples a small number of keys at random and then evicts the ones with the longest time since last access. LFU uses a probabilistic counter, called a Morris counter to estimate the object access frequency using just a few bits per object.

**LRU vs LFU:** both these algorithms aim to improve cache performance by keeping “hot” data in memory, but they prioritize different signals.
<div style="display:inline-block" />
    
| Policy |     Prioritizes     |                       Best for                      |              Strengths              |                    Tradeoffs                    |
|:------:|:-------------------:|:---------------------------------------------------:|:-----------------------------------:|:-----------------------------------------------:|
| LFU    | Frequency of access | Predictable or skewed workloads (e.g. best sellers) | Retains popular items               | More complex to tune; slower to adapt to change |
| LRU    | Recency of access   | Dynamic workloads (e.g. user sessions, dashboards)  | Simple to implement; adapts quickly | May evict frequently used but less recent data  |
</div>

[More details](https://redis.io/blog/lfu-vs-lru-how-to-choose-the-right-cache-eviction-policy/)

### Max Memory
Redis contains a configuration for maximum memory that can be used to cache data. This can be set to 0 to allow Redis to expand memory used as more data arrives. To view the current maximum memory set:
```bash
$ CONFIG GET maxmemory
8589934592
```

To set this value at runtime,
```bash
$ CONFIG SET maxmemory 0
```

Or set the `maxmemory` setting in Redis.conf.